### Load data

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

from lib.data_preparation import DataPreparation
from lib.preprocessing_data import Preprocessing


data_set = DataPreparation().load_data()
data_set = Preprocessing().lemmatization(data_set)

data_set = data_set.sample(frac=0.005).reset_index(drop=True)
data_set

---
# Bag Of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

import pandas as pd

processed_data = Preprocessing.tokenization(data_set)

text_data = processed_data['processed_tweet'].apply(lambda row: ' '.join(row))

bow_vector = vectorizer.fit_transform(text_data)


transformed_output = vectorizer.transform(text_data)
feature_names = vectorizer.get_feature_names_out()
dense_output = transformed_output.todense()
visulize_bow_vector = pd.DataFrame(
    dense_output, 
    columns=feature_names,
    index=text_data.index 
)

visulize_bow_vector


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# plt.figure(figsize=(15, 8))
plt.figure(figsize=(15, 8), facecolor='lightgray')
sns.heatmap(visulize_bow_vector, cmap="YlGnBu", annot=False, cbar=True)
plt.title("Bag-of-Words Model Heatmap")
plt.xlabel("Features (Words)")
plt.ylabel("Documents")
plt.show()

---
# 0 or 1, if the word exist (Binary/Count Vectorizer)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(binary=True)

import pandas as pd

processed_data = Preprocessing.tokenization(data_set)

text_data = processed_data['processed_tweet'].apply(lambda row: ' '.join(row))

binary_vector = vectorizer.fit_transform(text_data)

transformed_output = vectorizer.transform(text_data)
feature_names = vectorizer.get_feature_names_out()
dense_output = transformed_output.todense()
visulize_binary_vector = pd.DataFrame(
    dense_output, 
    columns=feature_names,
    index=text_data.index 
)

visulize_binary_vector


---
# Term Frequency-Inverse Document Frequency (TF-IDF)

In [ ]:
vectorizer = TfidfVectorizer()

processed_data = Preprocessing.tokenization(data_set)
text_data = processed_data['processed_tweet'].apply(lambda row: ' '.join(row))

tf_idf_vector = vectorizer.fit_transform(text_data)

transformed_output = vectorizer.transform(text_data)
feature_names = vectorizer.get_feature_names_out()
dense_output = transformed_output.todense()
visulize_tf_idf_vector = pd.DataFrame(
    dense_output, 
    columns=feature_names,
    index=text_data.index 
)
visulize_tf_idf_vector


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# plt.figure(figsize=(15, 8))
plt.figure(figsize=(15, 8), facecolor='lightgray')
sns.heatmap(visulize_tf_idf_vector, cmap="YlGnBu", annot=False, cbar=True)
plt.title("tf-idf Model Heatmap")
plt.xlabel("Features (Words)")
plt.ylabel("Documents")
plt.show()